Source: https://sec-api.io/resources/fama-french-factor-model
Data F-F_Research_Data_Factors.csv from: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip

In [26]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [27]:
port_monthly = pd.read_excel('IDX30 Top.xlsx',sheet_name='Performance')
port_monthly = port_monthly[['Date','Total Capital']]
port_monthly = port_monthly.set_index('Date',drop=True)

In [28]:
ff_factors_monthly = pd.read_excel('FF Portfolio Equal-Weight First-Day in Month.xlsx')
ff_factors_monthly = ff_factors_monthly.set_index('Date',drop=True)
# Filter factor dates to match the asset
ff_factors_subset = ff_factors_monthly[ff_factors_monthly.index.isin(port_monthly.index)].copy()
port_monthly= port_monthly[port_monthly.index.isin(ff_factors_subset.index)]
ff_factors_subset = ff_factors_subset[~ff_factors_subset.index.duplicated()]
# Step 3: Calculate excess returns for the asset
port_monthly['Portfolio Return'] = port_monthly['Total Capital'].pct_change()
port_monthly = port_monthly.drop(columns='Total Capital')
port_monthly = port_monthly.iloc[1:]
ff_factors_subset = ff_factors_subset.iloc[1:]
ff_factors_subset["Excess Return"] = port_monthly["Portfolio Return"] - ff_factors_subset["Rf"].astype('float')

In [32]:
# Prepare the independent variables (add a constant to the model)
#X = sm.add_constant(ff_factors_subset[["Mkt - Rf", "SMB", "HML", "MOM"]])
X = sm.add_constant(ff_factors_subset[['Mkt - Rf','SMB','HML','MOM']])
# The dependent variable
y = ff_factors_subset["Excess Return"]
X = X.apply(pd.to_numeric, errors='coerce')  # Convert all columns of X to numeric, turning non-numeric into NaN
y = pd.to_numeric(y, errors='coerce')       # Convert y to numeric
# Run the regression
model = sm.OLS(y, X).fit()
# Display the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          Excess Return   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     31.43
Date:                Sat, 24 May 2025   Prob (F-statistic):           6.54e-19
Time:                        19:05:39   Log-Likelihood:                 255.21
No. Observations:                 152   AIC:                            -500.4
Df Residuals:                     147   BIC:                            -485.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0748      0.004     18.333      0.0